In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [2]:
df = pd.read_csv('Algerian_forest_fires_dataset_UPDATE.csv',header=1)
df = df.drop([122,123],axis=0)
df.reset_index(inplace=True)
df=df.drop('index',axis=1)

In [3]:
df.head()

,day,month,year,Temperature,RH,Ws,Rain,FFMC,DMC,DC,ISI,BUI,FWI,Classes
0,01,06,2012,29,57,18,0,65.7,3.4,7.6,1.3,3.4,0.5,not fire
1,02,06,2012,29,61,13,1.3,64.4,4.1,7.6,1,3.9,0.4,not fire
2,03,06,2012,26,82,22,13.1,47.1,2.5,7.1,0.3,2.7,0.1,not fire
3,04,06,2012,25,89,13,2.5,28.6,1.3,6.9,0,1.7,0,not fire
4,05,06,2012,27,77,16,0,64.8,3,14.2,1.2,3.9,0.5,not fire


In [4]:
df['Region'] = ''
df[:122]=df[:122].replace('','Bejaia')
df[122:245]=df[122:245].replace('','Sidi-Bel')

In [5]:
df[df.isnull().any(axis=1)]
df = df.drop([165],axis=0)
df.reset_index(inplace=True)
df=df.drop('index',axis=1)

In [6]:
df.head()

,day,month,year,Temperature,RH,Ws,Rain,FFMC,DMC,DC,ISI,BUI,FWI,Classes,Region
0,01,06,2012,29,57,18,0,65.7,3.4,7.6,1.3,3.4,0.5,not fire,Bejaia
1,02,06,2012,29,61,13,1.3,64.4,4.1,7.6,1,3.9,0.4,not fire,Bejaia
2,03,06,2012,26,82,22,13.1,47.1,2.5,7.1,0.3,2.7,0.1,not fire,Bejaia
3,04,06,2012,25,89,13,2.5,28.6,1.3,6.9,0,1.7,0,not fire,Bejaia
4,05,06,2012,27,77,16,0,64.8,3,14.2,1.2,3.9,0.5,not fire,Bejaia


In [7]:
df.columns = df.columns.str.strip()

In [8]:
df['Classes'] = df['Classes'].str.strip()

In [9]:
one_hot = pd.get_dummies(df[['Region', 'Classes']])
# Drop column B as it is now encoded
df = df.drop(['Region', 'Classes'],axis = 1)
# Join the encoded df
df = df.join(one_hot)

In [10]:
df= df.drop(['Region_Sidi-Bel','Classes_fire'],axis = 1)

In [11]:
df.head()

,day,month,year,Temperature,RH,Ws,Rain,FFMC,DMC,DC,ISI,BUI,FWI,Region_Bejaia,Classes_not fire
0,01,06,2012,29,57,18,0,65.7,3.4,7.6,1.3,3.4,0.5,1,1
1,02,06,2012,29,61,13,1.3,64.4,4.1,7.6,1,3.9,0.4,1,1
2,03,06,2012,26,82,22,13.1,47.1,2.5,7.1,0.3,2.7,0.1,1,1
3,04,06,2012,25,89,13,2.5,28.6,1.3,6.9,0,1.7,0,1,1
4,05,06,2012,27,77,16,0,64.8,3,14.2,1.2,3.9,0.5,1,1


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243 entries, 0 to 242
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   day               243 non-null    object
 1   month             243 non-null    object
 2   year              243 non-null    object
 3   Temperature       243 non-null    object
 4   RH                243 non-null    object
 5   Ws                243 non-null    object
 6   Rain              243 non-null    object
 7   FFMC              243 non-null    object
 8   DMC               243 non-null    object
 9   DC                243 non-null    object
 10  ISI               243 non-null    object
 11  BUI               243 non-null    object
 12  FWI               243 non-null    object
 13  Region_Bejaia     243 non-null    uint8 
 14  Classes_not fire  243 non-null    uint8 
dtypes: object(13), uint8(2)
memory usage: 25.3+ KB


# Logistic Regression Model

In [21]:
#Classification
#spitting the data in dependent and independrnt sets
X = df.drop('Classes_not fire',axis=1)
y = df['Classes_not fire']

In [14]:
from sklearn.model_selection import train_test_split

In [22]:
#Spliting the datset in train and test data
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.30, random_state=42)

In [23]:
#Standardizing the data
from sklearn.preprocessing import StandardScaler

In [24]:
scaler = StandardScaler()

In [25]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [26]:
#Importing the model 
from sklearn.linear_model import LogisticRegression

In [27]:
Lor = LogisticRegression()

In [28]:
#Hperparameter tuning
from sklearn.model_selection import GridSearchCV

In [29]:
Parameters = {'penalty':['l1', 'l2', 'elasticnet'],'C':[1,2,3,4,5,8,20,34,60],'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],'max_iter':[100,200,300,500,1000]}

In [30]:
Classi_Lor = GridSearchCV(estimator=Lor,param_grid=Parameters,scoring='accuracy',cv=5)

In [31]:
Classi_Lor.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [1, 2, 3, 4, 5, 8, 20, 34, 60],
                         'max_iter': [100, 200, 300, 500, 1000],
                         'penalty': ['l1', 'l2', 'elasticnet'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']},
             scoring='accuracy')

In [32]:
Classi_Lor.best_params_

{'C': 1, 'max_iter': 100, 'penalty': 'l1', 'solver': 'liblinear'}

In [33]:
Classi_Lor.best_score_

0.9764705882352942

In [36]:
#Predicting the model

In [37]:
y_pred = Classi_Lor.predict(X_test)

In [38]:
#checking different metric of model

In [40]:
from sklearn.metrics import accuracy_score,classification_report

In [41]:
score = accuracy_score(y_pred,y_test)
print(score)

0.9726027397260274


In [42]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98        44
           1       0.97      0.97      0.97        29

    accuracy                           0.97        73
   macro avg       0.97      0.97      0.97        73
weighted avg       0.97      0.97      0.97        73

